This notebook creates a dict out of a dataframe of news : 
{'Company name' : dataframe of news containing the name of the company}

In [3]:
import pandas as pd
import numpy as np
import pickle
import os
%load_ext autoreload
%autoreload 2
%matplotlib inline
data_folder = '../data/'
df_to_read = 'Reuters' #Bloomberg or Reuters
df = pd.read_parquet(data_folder + 'financial_data' + df_to_read + '.parquet.gzip')


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


FileNotFoundError: ../data/financial_dataReuters.parquet.gzip

What is inside the dataframe :

In [2]:
print(df.columns)
print(f'{len(df)} entries')
df.head(5)


NameError: name 'df' is not defined

We drop everything not needed

In [63]:
df = df.drop(columns = ['Link', 'Headline', 'Journalists'])
df.head(3)

,Date,Article
0,"Fri Oct 20, 2006 6:15pm EDT","ANCHORAGE, Alaska (Reuters) - Exxon Mobil ( X..."
1,"Fri Oct 20, 2006 4:25pm EDT",SAN FRANCISCO/NEW YORK (Reuters) - Wall Stree...
2,"Sat Oct 21, 2006 2:21pm EDT",FRANKFURT (Reuters) - Internet service provid...


We get our dictionary (which is at first empty)

In [113]:
dict_name = "data_dict"
companies_to_get = ["google", "apple", "amazon", "facebook"]

f = open(dict_name+".pkl","r+b")
if (os.stat("./"+ dict_name +".pkl").st_size == 0):
    data_dict = {}
else :
    data_dict = pickle.load(f)
f.close()
keys = data_dict.keys()

#Check if the df we are going to add has corresponding variables (those given in 'cols')
cols = ['Date', 'Article']
assert(len(df.columns == len(cols))), 'Number of columns mismatch (needs 2 : Date and Article)'
for i in range(len(cols)):
    assert(cols[i] == df.columns.values[i]), f"Column mismatch : {df.columns.values[i]} when {cols(i)} was expected"

for company_name in companies_to_get:
    company_df = df[df.Article.apply(lambda article : company_name.lower() in article.lower())]
    if company_name in keys:
        data_dict[company_name] = data_dict[company_name].append(company_df)
    else :
        data_dict[company_name] = company_df

f = open(dict_name+".pkl","r+b")
pickle.dump(data_dict, f)
f.close()


In [108]:

f = open("data_dict.pkl","r+b")
data_dict2 = pickle.load(f)
print(data_dict2.keys())
f.close()

dict_keys(['google', 'amazon', 'apple'])


In [98]:
data_dict.keys()

dict_keys(['exxon', 'google', 'amazon', 'apple'])

Now we save everything in a py file

In [14]:
%%writefile make_dict.py
import pandas as pd
import numpy as np
import pickle
import os

def reset_dict(dict_dir):
    open(dict_dir, 'wb').close()

def get_df_news(data_folder, news_to_read, format_cols):
    df = pd.read_parquet(data_folder + 'financial_data' + news_to_read + '.parquet.gzip')
    df.rename(columns = {'Article':'Text', ''})
    for col in df.columns:
        if col not in format_cols:
            df = df.drop(columns=[col])
    return df

def get_dict(dict_dir):
    f = open(dict_dir,"r+b")
    if (os.stat(dict_dir).st_size == 0):
        data_dict = {}
    else :
        data_dict = pickle.load(f)
    f.close()
    return data_dict

def dump_dict(dct, dict_dir):
    f = open(dict_dir,"r+b")
    pickle.dump(dct, f)
    f.close()

def check_df_format(source_df, cols):
    assert(len(source_df.columns == len(cols))), 'Number of columns mismatch (needs 2 : Date and Article)'
    for i in range(len(cols)):
        assert(cols[i] == source_df.columns.values[i]), f"Column mismatch : {source_df.columns.values[i]} when {cols(i)} was expected"

        
def make_dict(source_df, companies_to_get, dict_dir, format_cols):
    data_dict = get_dict(dict_dir)
    keys = data_dict.keys()

    check_df_format(source_df, format_cols)
    companies_to_get = [company_name.lower() for company_name in companies_to_get]
    for company_name in companies_to_get:
        company_df = source_df[source_df.Article.apply(lambda article : company_name in article.lower())]
        if company_name in keys:
            data_dict[company_name] = pd.merge(data_dict[company_name], company_df,how='outer', on='Article')
        else :
            data_dict[company_name] = company_df
    dump_dict(data_dict, dict_dir)
    

Overwriting make_dict.py


Petit test des fonctions

In [12]:
from make_dict import *

data_folder = '../../data/'
news_to_read = 'Reuters'
format_cols=['Text', 'Date', 'Author']
dataf = get_df_news(data_folder, news_to_read, format_cols)

companies_to_get = ["Exxon", "Google", "Amazon"]
dict_dir = "./data_dict.pkl"
reset_dict(dict_dir)
make_dict(dataf, companies_to_get, dict_dir, format_cols)


In [15]:
print(get_dict(dict_dir)['exxon'].columns)

Index(['Text', 'Date', 'Author'], dtype='object')
